In [1]:
from __future__ import print_function

In [2]:
# We'll need numpy for some mathematical operations
import numpy as np

# matplotlib for displaying the output
import matplotlib.pyplot as plt
%matplotlib inline

# and IPython.display for audio output
import IPython.display

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display

In [9]:
audio_path = "/Users/Jasper/Music/iTunes/iTunes Media/Music/The White Stripes/Elephant/01 Seven Nation Army.mp3"

In [10]:
y, sr = librosa.load(audio_path)

/Users/Jasper/.local/share/virtualenvs/songShape-eCxQOFzy/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
